Cat or Dog Neural Network

In [21]:
#load modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import preprocessing
import tensorflow as tf

Image Processing

In [30]:
training_data_generator = ImageDataGenerator(
        #Rescale the image by 1/255 to normalize the pixel values
        rescale=1.0/255,
        #Randomly increase or decrease the size of the image by up to 20%
        #zoom_range=0.2, 
        #Randomly rotate the image between -15,15 degrees
        #rotation_range=15, 
        #Shift the image along its width by up to +/- 5%
        #width_shift_range=0.05, 
        #Shift the image along its height by up to +/- 5%
        #height_shift_range=0.05 
)

In [31]:
#variables being fed to training data generator
directory = "PetImages" #path to the folder containing the images
class_mode = "categorical"
color_mode = "rgb" 
target_size = (256, 256) #resizes each image to 256x256
batch_size = 32

In [32]:
training_iterator = training_data_generator.flow_from_directory(directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 25000 images belonging to 2 classes.


In [36]:
model = tf.keras.Sequential()

In [37]:
model.add(tf.keras.Input(shape = (256,256,3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100,activation="relu"))
model.add(tf.keras.layers.Dense(50,activation="relu"))
model.add(tf.keras.layers.Dense(2,activation="softmax"))

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 196608)            0         
                                                                 
 dense (Dense)               (None, 100)               19660900  
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 2)                 102       
                                                                 
Total params: 19,666,052
Trainable params: 19,666,052
Non-trainable params: 0
_________________________________________________________________


In [39]:
#model compilation
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [40]:
validation_data_generator = ImageDataGenerator( rescale=1./255)
validation_iterator = validation_data_generator.flow_from_directory(directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 25000 images belonging to 2 classes.


In [41]:
model.fit(
       training_iterator,
       steps_per_epoch=training_iterator.samples/batch_size,
       epochs=5,
       validation_data=validation_iterator,
       validation_steps=validation_iterator.samples/batch_size)

Epoch 1/5
  2/781 [..............................] - ETA: 2:05 - loss: 28.1629 - accuracy: 0.5156

UnknownError: Graph execution error:

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000019BA4768A90>
Traceback (most recent call last):

  File "c:\Users\Wiggles\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "c:\Users\Wiggles\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\Wiggles\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\Wiggles\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "c:\Users\Wiggles\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 1048, in generator_fn
    yield x[i]

  File "c:\Users\Wiggles\anaconda3\lib\site-packages\keras\preprocessing\image.p